In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
import sys
from retrying import retry
import re
import pyspark
import jieba.analyse
import jieba
import os 
from os import walk
from os import path
from sklearn.decomposition import PCA
from dask import delayed
import dask
import dask.dataframe as dd

In [2]:
read_time = time.time()
df = pd.read_excel('cluster_test.xlsx')
print(time.time()-read_time)

0.10073018074035645


In [3]:
# read_time = time.time()
# #df = pd.read_excel('cluster_test.xlsx')
# da_df = dd.read_csv('cluster_test.csv')
# print(time.time()-read_time)

In [4]:
# da_df.compute().head(3)

In [5]:
stopwords_path = 'stopwords.txt' # 停用词词表

In [6]:
def jiebaclearText(text):
    mywordlist = []
    seg_list = jieba.cut(text, cut_all=False)
    liststr="/ ".join(seg_list)
    f_stop = open(stopwords_path)

    with open('stopwords.txt', 'r', encoding='UTF-8') as f1:
        f_stop_text = f1.read()    # 读取数据

    
    f_stop_seg_list=f_stop_text.split('\n')
    for myword in liststr.split('/'):
        if not(myword.strip() in f_stop_seg_list) and len(myword.strip())>1:
            mywordlist.append(myword)
    return ''.join(mywordlist)

In [7]:
@delayed
def delayed_jiebaclearText(text):
    mywordlist = []
    seg_list = jieba.cut(text, cut_all=False)
    liststr="/ ".join(seg_list)
    f_stop = open(stopwords_path)

    with open('stopwords.txt', 'r', encoding='UTF-8') as f1:
        f_stop_text = f1.read()    # 读取数据

    
    f_stop_seg_list=f_stop_text.split('\n')
    for myword in liststr.split('/'):
        if not(myword.strip() in f_stop_seg_list) and len(myword.strip())>1:
            mywordlist.append(myword)
    return ''.join(mywordlist)

In [8]:
@delayed
def add(x, y):
    return x + ' ' + y

In [9]:
#x = da_df.compute()
x = df
y = df

In [10]:
y['contetn_after_jieba'] =1 
l = []
kai = time.time()
for i in range(len(x)):
    
    content = jiebaclearText(df.content	[i])
    title = jiebaclearText(df.分析对象	[i])
    c = content + ' ' + title
    l.append(c)
print(time.time() - kai)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\13544\AppData\Local\Temp\jieba.cache
Loading model cost 0.838 seconds.
Prefix dict has been built successfully.


9.849633932113647


In [11]:
# from dask.distributed import Client
# client = Client()
# client.cluster
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

In [12]:
x['contetn_after_jieba'] =1 
dask_l = []
kai = time.time()
for i in range(len(x)):
    content = delayed_jiebaclearText(df.content	[i])
    title = delayed_jiebaclearText(df.分析对象	[i])
    
    c = dask.delayed(add)(content, title)
    dask_l.append(c)
total_pic = dask.delayed(dask_l)
total = total_pic.compute(num_workers=2)
print(time.time() - kai)

7.537843942642212


In [16]:
# client.shutdown()

In [17]:
total[100]

'今日 发现 能代 CPA 能代 CPB 命运 Cle de Peau Beaute'

In [18]:
import joblib

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()

In [20]:
asd = v.fit_transform(total)

In [21]:
asd

<1000x12851 sparse matrix of type '<class 'numpy.float64'>'
	with 34320 stored elements in Compressed Sparse Row format>

In [22]:
tf_idf = asd.toarray()

In [23]:
tf_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
pca = PCA(n_components=800)
principalComponents = pca.fit_transform(tf_idf)

In [25]:
X = principalComponents

In [27]:
X

array([[-1.10701815e-01, -2.20372522e-01, -4.16628552e-02, ...,
        -1.43771053e-18,  1.75499306e-18,  1.98858429e-18],
       [-1.07649971e-01, -2.82332100e-01, -4.25626396e-02, ...,
        -1.43771053e-18,  1.75499306e-18,  1.98858429e-18],
       [-1.11652331e-01, -2.42609448e-01, -5.89410176e-02, ...,
        -1.43771053e-18,  1.75499306e-18,  1.98858429e-18],
       ...,
       [ 3.31562628e-01,  3.19071393e-02,  3.00826402e-02, ...,
        -1.43771053e-18,  1.75499306e-18,  1.98858429e-18],
       [-1.41762435e-01, -3.22262380e-01,  1.16267773e-01, ...,
        -1.43771053e-18,  1.75499306e-18,  1.98858429e-18],
       [-5.23170903e-01,  4.10586566e-01, -8.24494408e-02, ...,
        -1.43771053e-18,  1.75499306e-18,  1.98858429e-18]])

In [34]:
import dask_ml.cluster
km  = dask_ml.cluster.KMeans(n_clusters=10)

In [35]:
km.fit(X)

KMeans(n_clusters=10)

In [38]:
y = km.predict(principalComponents)

In [39]:
y

dask.array<astype, shape=(1000,), dtype=int32, chunksize=(125,), chunktype=numpy.ndarray>

In [41]:
len(y)

1000

In [44]:
for i in range(len(y)):
    if y[i] == 7:
        print(df.content[i])

#许佳琪的小猪仔[超话]#   0501求积分  #许佳琪幻乐森林# #许佳琪做家务的男人# #许佳琪清风朗月花正开# #许佳琪陈朗月# #许佳琪 skin##许佳琪的北漂日记##完美日记许佳琪##许佳琪 植村秀品牌挚友# #雪花秀品牌大使许佳琪##许佳琪 THE9门面主舞# #许佳琪 如意芳霏##许佳琪正能量偶像#   @THE9-许佳琪
#许佳琪的小猪仔[超话]#   0501积分  #许佳琪 skin#  #许佳琪清风朗月花正开#  #许佳琪幻乐森林#  #许佳琪 THE9门面主舞#  #雪花秀品牌大使许佳琪#  #完美日记许佳琪#  #许佳琪 植村秀品牌挚友#  #许佳琪正能量偶像# #许佳琪为歌而赞#  @THE9-许佳琪 ​
#许佳琪的小猪仔[超话]#   0501积分，指h  #许佳琪幻乐森林# #许佳琪陈朗月# #许佳琪清风朗月花正开# #许佳琪做家务的男人#  #许佳琪 skin# #许佳琪泊山城# #许佳琪傅宣#  #许佳琪 植村秀品牌挚友# #许佳琪正能量偶像#   #雪花秀品牌大使许佳琪# #完美日记许佳琪# @THE9-许佳琪 ​​​ ​​​​
#许佳琪的小猪仔[超话]#   求积分#许佳琪陈朗月##许佳琪 skin##许佳琪傅宣# #许佳琪如意芳霏# #许佳琪梁雪# #许佳琪平行迷途# #许佳琪小巨人运动会# #许佳琪正能量偶像# #雪花秀品牌大使许佳琪# #许佳琪 植村秀品牌挚友# #完美日记许佳琪# ​@THE9-许佳琪 ​
#许佳琪的小猪仔[超话]#  0501积分  #许佳琪 skin#  #许佳琪清风朗月花正开#  #许佳琪幻乐森林#  #许佳琪 THE9门面主舞#  #雪花秀品牌大使许佳琪#  #完美日记许佳琪#  #许佳琪 植村秀品牌挚友#  #许佳琪正能量偶像# #许佳琪为歌而赞#  @THE9-许佳琪 ​
#许佳琪的小猪仔[超话]#   许佳琪陈朗月#许佳琪陈朗月# 许佳琪清风朗月花正开#许佳琪清风朗月花正开# 许佳琪skin#许佳琪 skin# #许佳琪傅宣#  #许佳琪 植村秀品牌挚友# #许佳琪正能量偶像#   #雪花秀品牌大使许佳琪# #完美日记许佳琪# @THE9-许佳琪 ​
#许佳琪的小猪仔[超话]#   许佳琪陈朗月#许佳琪陈朗月# 许佳琪清风朗月花正开#许佳琪清风朗月花正开# 许佳琪skin#许佳琪 sk

In [42]:
x

,分析对象,content,情感,contetn_after_jieba
0,Make Up For Ever,天医归来秦羽 夏晓薇(免费/每费）阅读！ 看到野兔，立即放出猎鹰追捕在 木 头 上 ， ...,0,1
1,La Mer|Helena Rubinstein,顶级女海王“睡男人”有多野？日版项思醒被扒67张劲爆旧照，王思聪跪服：高端玩家！她，凭借67...,-1,1
2,La Mer,光希 眼部精华 | 眼周皮肤紧致后，眼角自然上提，真正做到了无痛眼部线雕！而且不长脂肪粒！人...,0,1
3,La Mer,@欣崽️创作的原声#无龄白挑战 @LA MER海蓝之谜 #全民任务 @DOU+小助手,0,1
4,La Mer,@湘妹心宝创作的原声#无龄白挑战 @LA MER海蓝之谜 #祝福大家五一劳动节快乐,1,1
...,...,...,...,...
995,Cle de Peau Beaute,#蔡徐坤[超话]#[心]#蔡徐坤代言cpb肌肤之钥# [心]#奔跑吧蔡徐坤# [心]#安慕希...,0,1
996,SK-II,出全新sk2大红瓶面霜80g的,0,1
997,Cle de Peau Beaute,#奔跑吧蔡徐坤# #蔡徐坤代言cpb肌肤之钥# 蔡徐坤奔跑吧兄弟蔡徐坤原创音乐制作人才...,1,1
998,Helena Rubinstein,#华晨宇和平精英品牌代言人# hcy 花花 #hr赫莲娜新生护肤代言人华晨宇# （组内...,0,1
